In [1]:
!pip install tensorflow==2.0.0-rc1 

  Using cached https://files.pythonhosted.org/packages/d0/33/376510eb8d6246f3c30545f416b2263eee461e40940c2a4413c711bdf62d/google_pasta-0.1.7-py3-none-any.whl
  Stored in directory: C:\Users\Yi\AppData\Local\pip\Cache\wheels\91\98\8d\10e3d4e04c959597a411b91acd3695e9e2d210e68ce3427aad
  Stored in directory: C:\Users\Yi\AppData\Local\pip\Cache\wheels\9a\1e\7a\456008eb5e47fd5de792c6139df6d5b3d5f71d51c6a0b94799
  Stored in directory: C:\Users\Yi\AppData\Local\pip\Cache\wheels\59\38\c6\234dc39b4f6951a0768fbc02d5b7207137a5b1d9094f0d54bf
  Stored in directory: C:\Users\Yi\AppData\Local\pip\Cache\wheels\7c\06\54\bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
Successfully built opt-einsum absl-py gast termcolor


In [3]:
!pip install tensorlayer

  Using cached https://files.pythonhosted.org/packages/66/7d/80daea9f4359253b76266bc3d4ebd665b5a56eab4117a96d60e63f2182cb/tensorlayer-2.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/14/df/12398d6bb3500f47904135a3d2459f644e5b71bade7a75ab72f107e032af/scikit_learn-0.21.0-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/7d/e3/20f3d364d6c8e5d2353c72a67778eb189176f08e873c9900e10c0287b84b/requests-2.21.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/fb/89/d90f9ff03285d8eb56994e8cec1b73a4d0dc9bb529c1f8e8e10b1b663843/progressbar2-3.39.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/eb/a0/19119d8b7c05be49baf6c593f11c432d571b70d805f2fe94c0585e55e4c8/python_utils-2.3.0-py2.py3-none-any.whl
  Found existing installation: scikit-learn 0.21.2
    Uninstalling scikit-learn-0.21.2:
      Successfully uninstalled scikit-learn-0.21.2
  Found existing installation: wrapt 1.11.2
    

ERROR: astroid 2.2.5 requires typed-ast>=1.3.0; implementation_name == "cpython", which is not installed.


In [1]:
import numpy as np
import tensorflow as tf
import tensorlayer as tl
tl.logging.set_verbosity(tl.logging.DEBUG)
# set gpu mem fraction or allow growth
# tl.utils.set_gpu_fraction()

In [2]:
# prepare data
X_train, y_train, X_val, y_val, X_test, y_test = tl.files.load_mnist_dataset(shape=(-1, 784))

# define the network
ni = tl.layers.Input([None, 784])
nn = tl.layers.Dropout(keep=0.8)(ni)
nn = tl.layers.Dense(n_units=800, act=tf.nn.relu)(nn)
nn = tl.layers.Dropout(keep=0.5)(nn)
nn = tl.layers.Dense(n_units=800, act=tf.nn.relu)(nn)
nn = tl.layers.Dropout(keep=0.5)(nn)
nn = tl.layers.Dense(n_units=10, act=None)(nn)
network = tl.models.Model(inputs=ni, outputs=nn, name="mlp")

[TL] Load or Download MNIST > data\mnist
[TL] data\mnist\train-images-idx3-ubyte.gz
[TL] data\mnist\t10k-images-idx3-ubyte.gz
[TL] Input  _inputlayer_1: [None, 784]
[TL] Dropout dropout_1: keep: 0.800000 
[TL] Dense  dense_1: 800 relu
[TL] Dropout dropout_2: keep: 0.500000 
[TL] Dense  dense_2: 800 relu
[TL] Dropout dropout_3: keep: 0.500000 
[TL] Dense  dense_3: 10 No Activation


In [3]:
X_train.shape

(50000, 784)

In [7]:
y_train.shape

(50000,)

In [4]:
# define metric.
def acc(_logits, y_batch):
    # return np.mean(np.equal(np.argmax(_logits, 1), y_batch))
    return tf.reduce_mean(
        tf.cast(tf.equal(tf.argmax(_logits, 1), tf.convert_to_tensor(y_batch, tf.int64)), tf.float32), name='accuracy'
    )


# print network information
print(network)

mlp(
  (_inputlayer_1): Input(shape=[None, 784], name='_inputlayer_1')
  (dropout_1): Dropout(keep=0.8, name='dropout_1')
  (dense_1): Dense(n_units=800, relu, in_channels='784', name='dense_1')
  (dropout_2): Dropout(keep=0.5, name='dropout_2')
  (dense_2): Dense(n_units=800, relu, in_channels='800', name='dense_2')
  (dropout_3): Dropout(keep=0.5, name='dropout_3')
  (dense_3): Dense(n_units=10, No Activation, in_channels='800', name='dense_3')
)


In [5]:
# train the network
tl.utils.fit(
    network, train_op=tf.optimizers.Adam(learning_rate=0.0001), cost=tl.cost.cross_entropy, X_train=X_train,
    y_train=y_train, acc=acc, batch_size=256, n_epoch=20, X_val=X_val, y_val=y_val, eval_train=True,
    tensorboard_dir='./tb_log'
)

# test
tl.utils.test(network, acc, X_test, y_test, batch_size=None, cost=tl.cost.cross_entropy)

# evaluation
_logits = tl.utils.predict(network, X_test)
y_pred = np.argmax(_logits, 1)
tl.utils.evaluation(y_test, y_pred, n_classes=10)

# save network weights
network.save_weights('model.h5')

# close tensorboard
# tl.utils.exit_tensorflow(port=6006)

[TL] Setting up tensorboard ...
[TL] [!] ./tb_log exists ...
[TL] Start training the network ...
[TL] Epoch 1 of 20 took 9.856574s
[TL]    train loss: 0.500460
[TL]    train acc: 0.839343
[TL] Epoch 5 of 20 took 11.320632s
[TL]    train loss: 0.268674
[TL]    train acc: 0.921354
[TL] Epoch 10 of 20 took 12.328689s
[TL]    train loss: 0.199035
[TL]    train acc: 0.942909
[TL] Epoch 15 of 20 took 10.796199s
[TL]    train loss: 0.158057
[TL]    train acc: 0.955349
[TL] Epoch 20 of 20 took 11.855987s
[TL]    train loss: 0.129489
[TL]    train acc: 0.963882
[TL] Total training time: 179.427649s
[TL] Start testing the network ...
[TL] confusion matrix: 
[[ 969    0    0    0    0    1    5    1    4    0]
 [   0 1127    2    2    0    1    2    0    1    0]
 [  10    1  982    6    3    0    6   11   13    0]
 [   0    0    7  974    0    7    0   10   11    1]
 [   1    4    3    0  942    0    9    1    2   20]
 [   6    1    0   19    1  836   12    2    9    6]
 [  10    3    0    0    5